# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096941


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:01<00:26,  1.06it/s]

Rendering models:  10%|█         | 3/30 [00:04<00:41,  1.54s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:50,  1.96s/it]

Rendering models:  17%|█▋        | 5/30 [00:08<00:37,  1.48s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:16<00:01,  2.84it/s]

Rendering models:  93%|█████████▎| 28/30 [00:17<00:00,  2.67it/s]

Rendering models:  97%|█████████▋| 29/30 [00:17<00:00,  2.44it/s]

Rendering models: 100%|██████████| 30/30 [00:17<00:00,  2.62it/s]

not-logged-in-3b0b929f4475fd276561      0.000246
not-logged-in-3b0b929f4475fd276561    328.451774
not-logged-in-673e227f920b24558cc1      0.001169
mwalsh868                               0.000344
jnarayanbvg                             0.001025
Jonas_Cross                             0.000199
awright5                                0.001240
not-logged-in-c6917d9b19f41e6c2995      0.005847
framosuw                                0.000718
jaksonA                                 0.005823
mmcdonough4                             0.003831
browny811                               0.000335
JoseA.G.v                               0.003318
crush202020                             0.001988
kanghan                                 0.001029
not-logged-in-dddbb97cfd84354416f5      0.001777
Saharisunshine                          0.002185
not-logged-in-214814e52004b42491a4      0.008777
chavez01                                0.000632
rhondacard                              0.001980
Lavadude            

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())